# Generate Gpt-2 Examples #

**prompt_text**:change the prompt to a text of your choice  
**response_length**: This is the maximum length of any response. It may choose to create shorter response itself. This cannot be longer than 1024  
**output_file**: This should be a .csv file. It can be downloaded an analyzed. If this is run multiple times, it will append all answers into the file.  
**num_of_responses**: This is usually a max of 4 for K80 GPU. It is possible that even at 4, the process may error. If this is the case rerun.  
  
**IN CASE OF ERROR**  
The process may error because the GPU gets filled up. Everytime you run this, make sure that the GPU starts with 0 memory used. The !Nvidia-smi command shows that.   
In the middle column it should say "0MB/11441MB"    
if it does not. The GPU needs to be cleared.  
Clear the GPU by clicking on the top menu "Kernel->Restart Kernel" Then rerun the notebook from the top cell.  

In [1]:

prompt_text="If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are"
#max reponse_length 1024
response_length=1000
#output_file will be created if it doesn't exist, otherwise answers will be appended
output_file="results.csv"
#max 4 (k80 gpu)
num_of_responses=20


In [2]:
#from ipyexperiments import *

In [3]:
!nvidia-smi


Wed May 13 14:10:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [4]:
all_seq=[]
results=[]

In [5]:
import torch
print(torch.__version__)
print(torch.cuda.device_count())

1.4.0
2


In [6]:
def return_results(tokenizer, encoded_prompt, outputs, all_seq):
    generated_sequences=[]
    total_sequence=""
    for generated_sequence_idx, generated_sequence in enumerate(outputs):
            print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
            generated_sequence = generated_sequence.tolist()
            #print(generated_sequence)
            # Decode text
            text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
            #print(text)
            # Remove all text after the stop token
            stop_token='<|endoftext|>'
            text = text[: text.find(stop_token) if stop_token else None]

            # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
            total_sequence = (
                prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
            )

            generated_sequences.append(total_sequence)
            print(total_sequence)
            all_seq.append(total_sequence)
    return

In [7]:
def run_gpt2(gpu, prompt_text,response_length,num_of_responses,all_seq):
    #%%time
    print('starting {} {} {}'.format(gpu,response_length, num_of_responses))
    import numpy as np
    from transformers import GPT2Tokenizer, GPT2LMHeadModel
    import torch
    n_gpu=torch.cuda.device_count()
    #device = xm.xla_device()
    device=torch.device(gpu)
    tokenizer = GPT2Tokenizer.from_pretrained('/spell/GPT2Model/GPT2Model/')
    model = GPT2LMHeadModel.from_pretrained('/spell/GPT2Model/GPT2Model/')
    #if n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
    #    model = torch.nn.DataParallel(model)
    model.to(device)
    model = torch.nn.DataParallel(model, device_ids=[0,1])
    encoded_prompt=tokenizer.encode(prompt_text, add_special_tokens=True,return_tensors="pt")
    encoded_prompt = encoded_prompt.to(device)

    outputs = model.module.generate(encoded_prompt,response_length,temperature=.8,num_return_sequences=num_of_responses,repetition_penalty=85,do_sample=True,top_k=80,top_p=.85 )
    if len(outputs.shape) > 2:
        outputs.squeeze_()
    return_results(tokenizer, encoded_prompt, outputs,all_seq)
    del model
    del encoded_prompt
    torch.cuda.empty_cache()
    return 

In [8]:
from concurrent.futures.thread import ThreadPoolExecutor
import concurrent.futures
gpus=torch.cuda.device_count()
gptresult=[]
print('gpus=',gpus)
with ThreadPoolExecutor(max_workers=gpus) as executor:
    totalthreads=int(num_of_responses/10)
    print('total runs=',totalthreads)
    for t in range(1,totalthreads+1):
        print('run number ',t)
        for i in range (gpus):
            print('sending to cuda:',str(i))
            #args=("cuda:"+str(i),prompt_text,response_length,num_of_responses,all_seq))
            gptresult.append(executor.submit(run_gpt2,"cuda:"+str(i),prompt_text,response_length,5,all_seq))
        for future in concurrent.futures.as_completed(gptresult):
            print('result=',future.result())
        gptresult=[]

print('all threads complete')

gpus= 2
total runs= 4
run number  1
sending to cuda: 0
starting cuda:0 1000 5
sending to cuda: 1
starting cuda:1 1000 5


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


=== GENERATED SEQUENCE 1 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are huge: it would mean we have no objective standards to measure religious ideas (which could include everything from atheism or agnosticism) against in order make them more like reality – i.e., less alienating?
The 'God of Abraham' theory was first proposed by Horkheimer with his book Dialectic Of Enlightenment, published on October 30th 1922. The idea came about because he felt there were too many gods outliving their usefulness so they had been abolished long ago but some still remain alive suchas Jesus who "was not able at one time nor another either directly …or indirectly" [Hornberger]." I believe," said Heinrich Heine referring back specifically againto those godless times when man's thoughts dominated over nature which allowed him control other living things…that our best days wi

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


=== GENERATED SEQUENCE 1 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are very serious indeed; it could mean we will never find any answers to questions of life's meaning or purpose in our universe again because there would no longer exist anything like 'God' outside ourselves (this was certainly one fear raised by critics at Kubrick when he made 2001: A Space Odyssey).


It has also been suggested [1]that these images may have had some psychological influence on him which affected his later work - especially since they were produced using highly sophisticated computers based upon human visual perception rather than seeing with their own eyes! This suggestion seems particularly implausible considering how much more obvious many other aspects about Dr Strangelove seem compared even nowto those used during filming.[2][3]. Another possibility put forward relat

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


=== GENERATED SEQUENCE 1 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are significant: it means you cannot believe in any religious doctrine without acknowledging there's more to our universe than what we see with five senses (and if they're correct then maybe other minds exist beyond ours too).
I'm not convinced of either point – I think science will eventually prove us wrong on both counts but since no one has yet done so why should believers try? Religion offers hope when nothing else does; whether or Not-God exists seems like such fundamental stuff these days anyway…
=== GENERATED SEQUENCE 2 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are quite profound; even if you believe in it on some level (and I do), there's no way to 

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


=== GENERATED SEQUENCE 1 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are terrifying:
"There will always exist some people who would believe in it to their dying day." ~ Stephen Hawking (from "The Grand Design")

 How about you? How many of these do we really need on our side before one more person realizes how stupid they've been being lately? It's not too late yet...
=== GENERATED SEQUENCE 2 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are profound."
The AI could also lead to more sophisticated moral systems than humans would have evolved with," he added in another interview on his blog. "Imagine how you'd feel if your child were diagnosed like [Machiavelli's] Prince when she was born? Imagine what it must've been at her birth

In [25]:
'''
import time
import nvgpu
keep_going=True
time.sleep(30)
while keep_going:
    time.sleep(5)
    n=nvgpu.gpu_info()
    #print(nvgpu.available_gpus())
    #print ("n=",n, len(n))
    for i in n:
        #print(i)
        print(i["index"],i["mem_used"],end=' ')
        if i["mem_used"]==0:
            keep_going=False
'''

'\nimport time\nimport nvgpu\nkeep_going=True\ntime.sleep(30)\nwhile keep_going:\n    time.sleep(5)\n    n=nvgpu.gpu_info()\n    #print(nvgpu.available_gpus())\n    #print ("n=",n, len(n))\n    for i in n:\n        #print(i)\n        print(i["index"],i["mem_used"],end=\' \')\n        if i["mem_used"]==0:\n            keep_going=False\n'

In [9]:
if len(all_seq)==len(set(all_seq)):
  print('no duplicates')
else:
  print('duplicates found')

no duplicates


In [10]:
import csv
import os
if os.path.exists(output_file):
    append_flag="a"
else: 
    append_flag="w"
with open (output_file, append_flag) as csvfile:
    writer=csv.writer(csvfile)
    for i in all_seq:
        writer.writerow([prompt_text, i])
    

In [11]:
print('run complete')

run complete
